# Теоретическая часть


### 1. Какие могут возникнуть проблемы, затрудняющие обучение модели, если не делать деление на $\sqrt d_k$ и работать с большим значением $d_{model}$ ?


При прямом проходе модели возникает такой момент, 
когда нам нужно найти **attention scores** от векторов $x_1, x_2, ..., x_n$ <br>
для некоторых векторов $y_1, y_2, ..., y_m$ (это могут быть те же вектора x в случае **self-attention**).
<br>
<br>
Рассмотрим, как будут считаться attention scores для одного вектора $y$ и матрицы векторов $X$.

$$att\_logits = y * X^T$$
$$att\_scores = softmax(att\_logits)$$

Изначально все вектора распределены одинаково $\sim N(0, 1)$
<br>
После скалярных произведений мы получаем, что компоненты вектора $att\_scores$ имеют мат.ожидание $= 0$ и дисперсию $= d$ (размерность векторов).<br>
**Пруфы:**
<br>
Рассмотрим два вектора $x, y \sim N(0, 1)$
$$(x, y) = \sum_{i=1}^{d} x_i * y_i$$

Т.к. величины независимо распределенные, то получаем, что:

$E\left[x_i * y_i\right] = E\left[x_i\right] * E\left[y_i\right] = 0 * 0 = 0$ 
<br>
$D\left[x_i * y_i\right] = D\left[x_i\right] * D\left[y_i\right] + E^2\left[x_i\right] * D\left[y_i\right] + E^2\left[y_i\right] * D\left[x_i\right] = 1 * 1 + 0 * 1 + 0 * 1 = 1$
<br>

И тогда выходит, что 
<br>
$E\left[(x, y)\right] = \sum_{i = 1}^{d} 0 = 0$
<br>
$D\left[(x, y)\right] = \sum_{i = 1}^{d} 1 = d$


Но оказывается, что если самая большая компонента софтмакса сильно больше, чем вторая по величине, то значения будут получаться очень маленькими и градиенты будут плохо протекать на предыдущие слои.
<br>
Именно это и может происходить в нашем случае, т.к. дисперсия становится в сотни раз больше -> разрывы между значениями скоров могут исчисляться десятками, что уже критично для софтмакса.

Но при домножении:
<br>
$D\left[\frac{1}{\sqrt d} * (x, y)\right] = \frac{1}{d} * D\left[(x, y)\right] = \frac{d}{d} = 1$

Получаем более адекватное распределение для софтмакса и всё должно работать хорошо.

### 2. В терминологии о-большого $O(·)$, зависящего от параметра $n$ опишите, чему равняется $path(1, n)$ для рекуррентного слоя и для MultiHeadAttention слоя. (Например, $O(n log(n)), O(n^2), O(1)$ )

#### 2.1 Посчитаем сначала для рекуррентного слоя.

Примем, что размеры эмбеддингов и скрытых слоёв одинаковые и равны **d**.

Внутри слоя в простейшем случае есть одна матрица преобразования $W$ размером $d \times d.$

На каждом таймстепе, мы перемножаем вектор $x_i$ и матрицу $W$ за $O(d^2)$ и применяем функцию активации за $O(d)$.

Т.к. таймстепов всего $n$, то получаем итоговую сложность $O(n * d^2)$. (т.к. зачастую $n$ даже меньше, чем $d$, то оставим его в итоговой сложности)

#### 2.2 Теперь посчитаем для MultiHeadAttention

(При подсчётах $d_v = d_k = d_q = \frac{d_{model}}{h}$)

Изначально у нас есть матрица $X[n \times d]$ и мы делаем из неё три матрицы $V, K, Q$ (просто копируем).

Затем мы применяем линейные преобразования для $V, K, Q$ за $O(n * d^2)$

Далее нам надо применить **attention**: перемножить $K * Q^T$ за $O(n^2 * d)$, применить софтмакс к результату за $O(n^2)$, и перемножить матрицу после софтмакса и матрицу $V$ за $O(n^2 * d)$.

Получаем итоговую сложность $O(n^2 * d + n * d^2)$

### 3. Опишите, какие типы масок используются?

Маски в attention нужны для того, чтобы при кодировании/декодировании мы не могли смотреть на некоторые слова.

Из примеров: 
1. При языковом моделировании в декодере используется маска в виде нижнетреугольной матрицы (выше главной диагонали нули, ниже единицы). Эта маска позволяет при предсказании очереднего слова не давать модели смотреть на следующие слова (т.к. иначе модель просто научиться выдавать это следующее слово на выход и всё обучение будет бессмысленно)

2. Т.к. большинство последовательностей разной длины, то их приходится дополнять паддингами до одинаковой длины.
И чтобы модель не использовала эти бесполезные паддинги в рассчетах attention их тоже нужно маскировать.

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True